# Project Part 2
## Question 1
### Imports and data wrangling


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq
from scipy.optimize import least_squares
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

In [2]:
swaption = pd.read_excel("./data/IR Data.xlsx",     # load swaption pricing
                sheet_name = 'Swaption',
                header=2)
swaption[swaption.columns[2:]] /= 100   # scale percentage IV to fraction
fsr = pd.read_csv('./data/fsr.csv')     # load forward swap rates
ois = pd.read_csv('./data/ois.csv')     # load discounting curve

In [3]:
swaption.head()

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,0.9157,0.6203,0.44130,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,0.30297
1,1Y,2Y,0.8327,0.6124,0.46570,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,0.33523
2,1Y,3Y,0.7392,0.5687,0.44770,0.35745,0.32317,0.2978,0.2829,0.2780,0.2877,0.30725,0.32833
3,1Y,5Y,0.5519,0.4464,0.36510,0.30242,0.27851,0.2607,0.2498,0.2456,0.2512,0.26536,0.28165
4,1Y,10Y,0.4118,0.3504,0.30207,0.26619,0.25351,0.2447,0.2398,0.2382,0.2425,0.25204,0.26355


In [4]:
fsr.head()

,start,tenor,f_irs,f_ois
0,1,1,0.031922,0.003495
1,1,2,0.033217,0.003520
2,1,3,0.033982,0.003662
3,1,5,0.035237,0.003997
4,1,10,0.038406,0.004734


In [5]:
ois.head()

,years,Rate,df,Tenor,Product,f,cumsum_df
0,0.5,0.00250,0.998752,6m,OIS,0.002498,0.998752
1,1.0,0.00300,0.997009,1y,OIS,0.003493,1.995761
2,1.5,NaN,0.995270,NaN,NaN,NaN,2.991030
3,2.0,0.00325,0.993531,2y,OIS,0.003495,3.984561
4,2.5,NaN,0.991773,NaN,NaN,NaN,4.976334


In [6]:
bps = [-.0200, -.0150, -.0100, -.0050, -.0025, 0, .0025, .0050, .0100, .0150, .0200]
df_strike = swaption.copy()
for i,forward_rate in enumerate(fsr.f_irs):
    for j, bp in enumerate(bps):
        df_strike.iloc[i,j+2] = forward_rate + bp

df_strike.head()

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,0.011922,0.016922,0.021922,0.026922,0.029422,0.031922,0.034422,0.036922,0.041922,0.046922,0.051922
1,1Y,2Y,0.013217,0.018217,0.023217,0.028217,0.030717,0.033217,0.035717,0.038217,0.043217,0.048217,0.053217
2,1Y,3Y,0.013982,0.018982,0.023982,0.028982,0.031482,0.033982,0.036482,0.038982,0.043982,0.048982,0.053982
3,1Y,5Y,0.015237,0.020237,0.025237,0.030237,0.032737,0.035237,0.037737,0.040237,0.045237,0.050237,0.055237
4,1Y,10Y,0.018406,0.023406,0.028406,0.033406,0.035906,0.038406,0.040906,0.043406,0.048406,0.053406,0.058406


In [7]:
# ATM Black IV
swaption.head(2)

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,0.9157,0.6203,0.4413,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,0.30297
1,1Y,2Y,0.8327,0.6124,0.4657,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,0.33523


## Valuing a swaption
$$
\text{Payer Swaption} = \left[ P_{n+1,N}(T) \left( S_{n,N}(T) - K \right) \right]^+
$$
$$
\text{Receiver Swaption} = \left[ P_{n+1,N}(T) \left( K - S_{n,N}(T) \right) \right]^+
$$

Using PVBP as risk neutral numeraire, and applying Black normal model
$$
\begin{aligned}
\frac{V^{\text{payer}}_{n,N} (0)}{P_{n+1,N} (0)} &= \mathbb{E}^{n+1,N} \left[ \frac{V^{\text{payer}}_{n,N} (T_n)}{P_{n+1,N} (T_n)} \right] \\
\Rightarrow V^{\text{payer}}_{n,N} (0) &= P_{n+1,N} (0) \mathbb{E}^{n+1,N} \left[ (S_{n,N} (T) - K)^+ \right] \\ \\
V^{\text{payer}}_{n,N} (0) = & P_{n+1,N} (0) \left[ S_{n,N} (0) \Phi(d_1) - K \Phi(d_2)  \right]
\end{aligned}
$$
1. get 

In [8]:
ois.head()

,years,Rate,df,Tenor,Product,f,cumsum_df
0,0.5,0.00250,0.998752,6m,OIS,0.002498,0.998752
1,1.0,0.00300,0.997009,1y,OIS,0.003493,1.995761
2,1.5,NaN,0.995270,NaN,NaN,NaN,2.991030
3,2.0,0.00325,0.993531,2y,OIS,0.003495,3.984561
4,2.5,NaN,0.991773,NaN,NaN,NaN,4.976334


In [ ]:
def pvbp(expiry, tenor):
    # Calculate PVBP
    """
    Return PVBP for a period from expiry to end of tenor in years t should be >= 0.5
    assumed to be semi-annual payments
    """
    t = expiry
    T = expiry + tenor
    # TODO ask prof if it is discounted by OIS and if semi-annual or annual payments
    delta_t = 0.5       # for semi-annual payments
    sum_df = (ois.loc[(ois.years > t) & (ois.years <= T)]).df.sum()

    return delta_t * sum_df
    
def black76_pay(F, K, expiry, tenor, sigma):
    """
    Return value of payer swaption via Black Normal model and multiplied by PVBP
    parameters
        F: forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * sigma * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = F * norm.cdf(d_1) - K * norm.cdf(d_2)
    return pvbp(expiry,tenor) * black_option
    
def black76_rec(F, K, expiry, tenor, sigma):
    """
    Return value of receiver swaption via Black Normal model and multiplied by PVBP
    parameters
        F: Forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * sigma * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = K * norm.cdf(-d_2) - F * norm.cdf(-d_1)
    return pvbp(expiry,tenor) * black_option


pvbp(1,5)

4.935898972941314

In [10]:
swaption.head()

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,0.9157,0.6203,0.44130,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,0.30297
1,1Y,2Y,0.8327,0.6124,0.46570,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,0.33523
2,1Y,3Y,0.7392,0.5687,0.44770,0.35745,0.32317,0.2978,0.2829,0.2780,0.2877,0.30725,0.32833
3,1Y,5Y,0.5519,0.4464,0.36510,0.30242,0.27851,0.2607,0.2498,0.2456,0.2512,0.26536,0.28165
4,1Y,10Y,0.4118,0.3504,0.30207,0.26619,0.25351,0.2447,0.2398,0.2382,0.2425,0.25204,0.26355


In [19]:
fsr.shape, df_strike.shape, swaption.shape

((15, 4), (15, 13), (15, 13))

In [29]:
row_idx = 1
col_idx = 5
strike_header = swaption.columns[2:][col_idx]
F = fsr.f_irs[row_idx]
K = df_strike[strike_header][row_idx]
expiry = fsr['start'][row_idx]
tenor = fsr['tenor'][row_idx]
sigma = swaption['ATM'][row_idx]

black76_pay(F, K, expiry, tenor, sigma)

0.007069578821992686

In [ ]:
def dd_pay(F, K, expiry, tenor, sigma, beta, payoff):
    """
    Return value of receiver swaption  via Displaced Diffusion model and multiplied by PVBP
    parameters
        F: forwar par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
        beta: DD beta
    """
    return black76_pay(F/beta, K + (1-beta)/beta*F, expiry, tenor, sigma * beta)

,Expiry,Tenor,-200bps,-150bps,-100bps,-50bps,-25bps,ATM,+25bps,+50bps,+100bps,+150bps,+200bps
0,1Y,1Y,0.011922,0.016922,0.021922,0.026922,0.029422,0.031922,0.034422,0.036922,0.041922,0.046922,0.051922
1,1Y,2Y,0.013217,0.018217,0.023217,0.028217,0.030717,0.033217,0.035717,0.038217,0.043217,0.048217,0.053217
2,1Y,3Y,0.013982,0.018982,0.023982,0.028982,0.031482,0.033982,0.036482,0.038982,0.043982,0.048982,0.053982
3,1Y,5Y,0.015237,0.020237,0.025237,0.030237,0.032737,0.035237,0.037737,0.040237,0.045237,0.050237,0.055237
4,1Y,10Y,0.018406,0.023406,0.028406,0.033406,0.035906,0.038406,0.040906,0.043406,0.048406,0.053406,0.058406
5,5Y,1Y,0.019258,0.024258,0.029258,0.034258,0.036758,0.039258,0.041758,0.044258,0.049258,0.054258,0.059258
6,5Y,2Y,0.020059,0.025059,0.030059,0.035059,0.037559,0.040059,0.042559,0.045059,0.050059,0.055059,0.060059
7,5Y,3Y,0.020045,0.025045,0.030045,0.035045,0.037545,0.040045,0.042545,0.045045,0.050045,0.055045,0.060045
8,5Y,5Y,0.021060,0.026060,0.031060,0.036060,0.038560,0.041060,0.043560,0.046060,0.051060,0.056060,0.061060
9,5Y,10Y,0.023566,0.028566,0.033566,0.038566,0.041066,0.043566,0.046066,0.048566,0.053566,0.058566,0.063566


## Calibrating Beta
- One beta is calculated for one expiry and tenor, across multiple strikes. DD model is used to have the least errors against the implied vols for all given strikes.
- To calibrate for beta, use a fitting algorithm to solve for a beta that will reduce the objective function $\sigma_{implied} = \sigma_{black}$, where $\sigma_{dd} = \sigma_{black}\beta$

steps:
1. calculate DD model swaption price
2. calculate $\sigma_{black}$ based on DD model swaption price
3. conduct beta search by reducing error as $\sigma_{implied} - \sigma_{black}$ by changing DD $\beta$

In [ ]:
    beta = DD_fit_beta(df["K"], df["ivol"], par_rate, swap_data1["Expiry"][i], swap_data1["Tenor"][i], swap_data1["ATM"][i]/100, df["payoff"])


In [32]:
a = swaption['ATM'] 
b = swaption['ATM'] 
if a> b:
    print('in')
else:
    print('out')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Displaced-Diffusion Model
Assumes a linear combination of Geometric and Arithmetic Brownian Process to model swap rates